In [1]:
import pandas

train_data = pandas.read_csv('train.csv', sep=',', header=0, encoding='latin-1')
test_data = pandas.read_csv('test.csv', sep=',', header=0, encoding='latin-1')

train_data = train_data.drop(['ItemID'], axis=1)

print(len(train_data))
train_data.head()

99989


,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [2]:
item_id_data = test_data.ItemID

In [3]:
import re

def norm1(row):
    return re.sub("[-()\"#/@;:<>{}`+=~|.!?,]",'', row['SentimentText'])
    
train_data['norm'] = train_data.apply(norm1, axis=1)
test_data['norm'] = test_data.apply(norm1, axis=1)

train_data.head()

,Sentiment,SentimentText,norm
0,0,is so sad for my APL frie...,is so sad for my APL friend
1,0,I missed the New Moon trail...,I missed the New Moon trailer
2,1,omg its already 7:30 :O,omg its already 730 O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...,Omgaga Im sooo im gunna CRy I've b...
4,0,i think mi bf is cheating on me!!! ...,i think mi bf is cheating on me ...


In [4]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
norm = train_data['norm']
cv.fit(norm)

x = cv.transform(norm)
y = train_data[['Sentiment']]

x.shape

(99989, 117073)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y)

num_classes = 2

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [6]:
import tensorflow as tf
from keras import models
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers import Embedding

# embedding_vector_length = 16 
# model = models.Sequential() 
# model.add(Embedding(5000, embedding_vector_length, input_length=9844)) 
# model.add(LSTM(100))
# model.add(Flatten()) 
# model.add(Dense(num_classes, activation='sigmoid'))

model = models.Sequential()
model.add(Dense(100, activation='tanh', kernel_initializer='uniform', input_shape=(117073,)))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh', kernel_initializer='uniform'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               11707400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 11,717,702
Trainable params: 11,717,702
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [8]:
results = model.fit(
 x_train, y_train,
 epochs = 2,
 batch_size = 256,
 validation_data = (x_test, y_test)
)

Train on 74991 samples, validate on 24998 samples
Epoch 1/2
74991/74991 [==============================] - 142s 2ms/step - loss: 0.5248 - acc: 0.7400 - val_loss: 0.4953 - val_acc: 0.7665
Epoch 2/2
74991/74991 [==============================] - 140s 2ms/step - loss: 0.3554 - acc: 0.8494 - val_loss: 0.5396 - val_acc: 0.7485


In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5396335315814981
Test accuracy: 0.7484598767901433


In [10]:
# t_pred = model.predict(T);
# pandas.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': t_pred}).to_csv('prediction.csv', index=False)